In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas as pd

In [ ]:
file_path = '/content/data_transporte_bd.csv'
data = pd.read_csv(file_path)
data

,ID_Viaje,Fecha,Ruta,Numero_Pasajeros,Duracion_Viaje_Minutos,Retraso_Minutos,Tipo_Transporte,Region,Dia_Semana
0,3,2023-01-01 00:02:00,Madrid - Zaragoza,10,94.0,119,Autobús,Madrid,Sunday
1,7,2023-01-01 00:06:00,Madrid - Malaga,78,608.0,21,Autobús,Malaga,Sunday
2,11,2023-01-01 00:10:00,Valencia - Sevilla,67,345.0,2,Autobús,Sevilla,Sunday
3,19,2023-01-01 00:18:00,Zaragoza - Barcelona,27,356.0,24,Autobús,Zaragoza,Sunday
4,34,2023-01-01 00:33:00,Malaga - Sevilla,90,362.0,37,Autobús,Sevilla,Sunday
...,...,...,...,...,...,...,...,...,...
14229,99932,2023-03-11 09:31:00,Zaragoza - Barcelona,14,404.0,10,Autobús,Zaragoza,Saturday
14230,99942,2023-03-11 09:41:00,Bilbao - Madrid,77,444.0,3,Tren,Bilbao,Saturday
14231,99991,2023-03-11 10:30:00,Madrid - Barcelona,11,623.0,66,Autobús,Madrid,Saturday
14232,99993,2023-03-11 10:32:00,Madrid - Barcelona,14,98.0,14,Tren,Barcelona,Saturday


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14234 entries, 0 to 14233
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID_Viaje                14234 non-null  int64  
 1   Fecha                   14234 non-null  object 
 2   Ruta                    14234 non-null  object 
 3   Numero_Pasajeros        14234 non-null  int64  
 4   Duracion_Viaje_Minutos  14234 non-null  float64
 5   Retraso_Minutos         14234 non-null  int64  
 6   Tipo_Transporte         14234 non-null  object 
 7   Region                  14234 non-null  object 
 8   Dia_Semana              14234 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 1001.0+ KB


In [ ]:
# Convertir fecha a datetime
data['Fecha'] = pd.to_datetime(data['Fecha'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14234 entries, 0 to 14233
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ID_Viaje                14234 non-null  int64         
 1   Fecha                   14234 non-null  datetime64[ns]
 2   Ruta                    14234 non-null  object        
 3   Numero_Pasajeros        14234 non-null  int64         
 4   Duracion_Viaje_Minutos  14234 non-null  float64       
 5   Retraso_Minutos         14234 non-null  int64         
 6   Tipo_Transporte         14234 non-null  object        
 7   Region                  14234 non-null  object        
 8   Dia_Semana              14234 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 1001.0+ KB


In [ ]:
# Filtrar los datos anómalos en Duracion_Viaje_Minutos y Retraso_Minutos
data = data[(data['Duracion_Viaje_Minutos'] > 120) & (data['Retraso_Minutos'] < 999)]

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12086 entries, 1 to 14233
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   ID_Viaje                12086 non-null  int64         
 1   Fecha                   12086 non-null  datetime64[ns]
 2   Ruta                    12086 non-null  object        
 3   Numero_Pasajeros        12086 non-null  int64         
 4   Duracion_Viaje_Minutos  12086 non-null  float64       
 5   Retraso_Minutos         12086 non-null  int64         
 6   Tipo_Transporte         12086 non-null  object        
 7   Region                  12086 non-null  object        
 8   Dia_Semana              12086 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(4)
memory usage: 944.2+ KB


In [ ]:
# Guardar el DataFrame en un archivo CSV
file_path = '/content/data_final_db.csv'
data.to_csv(file_path, index=False)

In [ ]:
import sqlite3

# Conectar a la base de datos SQLite (o crear una nueva)
conn = sqlite3.connect('transporte_españa.db')
c = conn.cursor()

# Crear la tabla
c.execute('''
CREATE TABLE IF NOT EXISTS transporte_publico (
    ID_Viaje INTEGER PRIMARY KEY,
    Fecha DATETIME,
    Ruta TEXT,
    Numero_Pasajeros INTEGER,
    Duracion_Viaje_Minutos FLOAT,
    Retraso_Minutos INTEGER,
    Tipo_Transporte TEXT,
    Region TEXT,
    Dia_Semana TEXT
)
''')

conn.commit()

In [ ]:
import pandas as pd

# Cargar los datos desde el archivo CSV
data = pd.read_csv('/content/data_final_db.csv')

# Convertir la columna Fecha a formato datetime
# data['Fecha'] = pd.to_datetime(data['Fecha'])

# Preparar los datos para la inserción
data_to_insert = data.values.tolist()

# Insertar los datos en la tabla
c.executemany('''
INSERT INTO transporte_publico
(ID_Viaje, Fecha, Ruta, Numero_Pasajeros, Duracion_Viaje_Minutos, Retraso_Minutos, Tipo_Transporte, Region, Dia_Semana)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
''', data_to_insert)

# Confirmar los cambios y cerrar la conexión
conn.commit()
conn.close()


In [ ]:
# Conectar a la base de datos SQLite
conn = sqlite3.connect('transporte_españa.db')
c = conn.cursor()

# Ejecutar una consulta SQL
c.execute('SELECT * FROM transporte_publico LIMIT 5')

# Obtener los resultados
rows = c.fetchall()

In [ ]:
# Convirtiendo los resultados en un DataFrame de pandas
columns = [desc[0] for desc in c.description]
df = pd.DataFrame(rows, columns=columns)

# Mostrar el DataFrame
print(df)

   ID_Viaje                Fecha                  Ruta  Numero_Pasajeros  \
0         7  2023-01-01 00:06:00       Madrid - Malaga                78   
1        11  2023-01-01 00:10:00    Valencia - Sevilla                67   
2        19  2023-01-01 00:18:00  Zaragoza - Barcelona                27   
3        34  2023-01-01 00:33:00      Malaga - Sevilla                90   
4        36  2023-01-01 00:35:00       Bilbao - Madrid                34   

   Duracion_Viaje_Minutos  Retraso_Minutos Tipo_Transporte    Region  \
0                   608.0               21         Autobús    Malaga   
1                   345.0                2         Autobús   Sevilla   
2                   356.0               24         Autobús  Zaragoza   
3                   362.0               37         Autobús   Sevilla   
4                   264.0               24         Autobús    Madrid   

  Dia_Semana  
0     Sunday  
1     Sunday  
2     Sunday  
3     Sunday  
4     Sunday  


# Preparo datos para Dashboard en Power BI

In [ ]:
data.head()

,ID_Viaje,Fecha,Ruta,Numero_Pasajeros,Duracion_Viaje_Minutos,Retraso_Minutos,Tipo_Transporte,Region,Dia_Semana
0,7,2023-01-01 00:06:00,Madrid - Malaga,78,608.0,21,Autobús,Malaga,Sunday
1,11,2023-01-01 00:10:00,Valencia - Sevilla,67,345.0,2,Autobús,Sevilla,Sunday
2,19,2023-01-01 00:18:00,Zaragoza - Barcelona,27,356.0,24,Autobús,Zaragoza,Sunday
3,34,2023-01-01 00:33:00,Malaga - Sevilla,90,362.0,37,Autobús,Sevilla,Sunday
4,36,2023-01-01 00:35:00,Bilbao - Madrid,34,264.0,24,Autobús,Madrid,Sunday


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12086 entries, 0 to 12085
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ID_Viaje                12086 non-null  int64  
 1   Fecha                   12086 non-null  object 
 2   Ruta                    12086 non-null  object 
 3   Numero_Pasajeros        12086 non-null  int64  
 4   Duracion_Viaje_Minutos  12086 non-null  float64
 5   Retraso_Minutos         12086 non-null  int64  
 6   Tipo_Transporte         12086 non-null  object 
 7   Region                  12086 non-null  object 
 8   Dia_Semana              12086 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 849.9+ KB


In [ ]:
# Convertiendo fecha a datetime
data['Fecha'] = pd.to_datetime(data['Fecha'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
# Reemplazar punto decimal por coma, para evitar conflicto en Power BI
data['Duracion_Viaje_Minutos'] = data['Duracion_Viaje_Minutos'].apply(lambda x: str(x).replace('.', ','))

In [ ]:
data.head()

,ID_Viaje,Fecha,Ruta,Numero_Pasajeros,Duracion_Viaje_Minutos,Retraso_Minutos,Tipo_Transporte,Region,Dia_Semana
0,7,2023-01-01 00:06:00,Madrid - Malaga,78,"608,0",21,Autobús,Malaga,Sunday
1,11,2023-01-01 00:10:00,Valencia - Sevilla,67,"345,0",2,Autobús,Sevilla,Sunday
2,19,2023-01-01 00:18:00,Zaragoza - Barcelona,27,"356,0",24,Autobús,Zaragoza,Sunday
3,34,2023-01-01 00:33:00,Malaga - Sevilla,90,"362,0",37,Autobús,Sevilla,Sunday
4,36,2023-01-01 00:35:00,Bilbao - Madrid,34,"264,0",24,Autobús,Madrid,Sunday


In [ ]:
# Guardando el DataFrame en un archivo CSV
file_path = '/content/data_transporte_final.csv'
data.to_csv(file_path, index=False)